In [ ]:
import numpy as np
import pyvista as pv
from tqdm import tqdm

from kale import Engine, save_movie, show_ui
from kale.algorithms import (
    contour_banded,
    scalars_operation_algorithm,
)
from kale import theme

## Load Data

Set up the driver of the data IO and time step management with the `kale.Engine` class

In [ ]:
# MESH_GEOMETRY_FILE_NAME = "nankai_very_very_dense.vtk"
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
# MESH_VALUES_FILE_NAME = "2023_03_03_16_15_24.hdf"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.01
TIME_STEP_INTERESTING = 55849
TIME_STEP_INTERESTING = 1000

In [ ]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

In [ ]:
# Set which array to be active
engine.mesh.active_scalars_name = "geometric_moment"
engine.time_step = TIME_STEP_INTERESTING

In [ ]:
var = "geometric_moment"
engine.mesh.active_scalars_name = var

# Updated video style

In [ ]:
# Manipulate the quantity
operation = np.cbrt
MANIPULATED_ARRAY_NAME = f"cbrt({engine.mesh.active_scalars_name})"
manipulated = scalars_operation_algorithm(
    engine.algorithm, operation, output_scalars_name=MANIPULATED_ARRAY_NAME
)

# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
CONTOUR_LEVELS = np.linspace(-600, 600, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]
contour, edges = contour_banded(
    manipulated,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
)

pl = pv.Plotter()
pl.add_mesh(engine.boundary_smoothed)

actor = pl.add_mesh(
    contour,
    # cmap=COLOR_MAPS[var],
    # cmap="PuOr_r",
    cmap="RdYlBu_r",
    clim=CLIM,
    n_colors=N_COLORS,
)
pl.add_scalar_bar(mapper=actor.mapper, title=f"", **theme.SCALAR_BAR_OPTS)

pl.add_mesh(edges)
pl.add_mesh(engine.boundary)

box = pv.Box(engine.mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")

pl.camera.zoom(1.2)

# Make a video over time steps
step = 500
pl.open_movie(f"{var}_nankai_movie_only.mp4", framerate=60, quality=5)
for tindex in tqdm(range(0, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(f"i = {tindex:05d}", name="time-step-label", position="lower_edge")
    pl.write_frame()
pl.close()